[Multi Label BERT Fine-Tuning](https://curiousily.com/posts/multi-label-text-classification-with-bert-and-pytorch-lightning/)

[Multi Label distilBERT Fine-Tuning](https://medium.com/analytics-vidhya/finetune-distilbert-for-multi-label-text-classsification-task-994eb448f94c)

[Another Multi Label Fine Tuning this time in transformer without additional layer](https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://huggingface.co/blog/Valerii-Knowledgator/multi-label-classification&ved=2ahUKEwiCl8GW6uSFAxVag_0HHeiCBjkQFnoECB8QAQ&usg=AOvVaw0yjiyuFTfhVdUPRWVtAOXA)

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install -U transformers datasets lightning wandb

In [3]:
# parameters

epochs = 10
learning_rate = 1e-5
max_length = 500
batch_size = 32
num_workers = 15
train_sample = 'balanced toxic data set'
model_name = 'distilbert-base-uncased'
task_folder= "Multi Label Hate Speech Bullying NSFW"
LABEL_COLUMNS = ["nsfw", "hate_speech", "bullying"]
n_labels = len(LABEL_COLUMNS)
id2label=dict(zip(range(n_labels), LABEL_COLUMNS))
label2id=dict(zip(LABEL_COLUMNS, range(n_labels)))
wandb_name = f"kaggle - {model_name} - multi label transformer - train sample {train_sample} - epochs {epochs} - batch size {batch_size} - max_length {max_length} learning rate {str(learning_rate)} num_workers {num_workers}_with_all_accuracy"
wandb_project = task_folder
pickle_name = f"kaggle - {model_name}_multi_label_transformer_train_sample_{train_sample}_epochs_{epochs}_batch_size_{batch_size}_max_length_{max_length}_learning_rate_{str(learning_rate)}_num_workers_{num_workers}_with_all_accuracy"

In [4]:
import wandb

wandb.init(project=wandb_project, name=wandb_name, group="yoav_yosef")

wandb: Currently logged in as: borodache (yoav_yosef). Use `wandb login --relogin` to force relogin


In [5]:
import pandas as pd

df_train = pd.read_csv("/content/drive/MyDrive/BIU - Final Project/Data/kaggle_train_toxic_final_balanced.csv")
df_val = pd.read_csv("/content/drive/MyDrive/BIU - Final Project/Data/kaggle_val_toxic_final_balanced.csv")
df_test = pd.read_csv("/content/drive/MyDrive/BIU - Final Project/Data/kaggle_test_toxic_final_balanced.csv")

In [6]:
sum(df_train[LABEL_COLUMNS].value_counts())

7103

In [7]:
df_train["len_text"] = df_train["text"].apply(lambda text: len(text.split()))
print(df_train.len_text.max(), df_train.len_text.mean())
df_val["len_text"] = df_val["text"].apply(lambda text: len(text.split()))
print(df_val.len_text.max(), df_val.len_text.mean())
df_test["len_text"] = df_test["text"].apply(lambda text: len(text.split()))
print(df_test.len_text.max(), df_test.len_text.mean())

765 28.552724201041812
583 29.1316685584563
702 27.480681818181818


In [8]:
X_train, y_train, X_val, y_val, X_test, y_test = df_train["text"], df_train[LABEL_COLUMNS], df_val["text"], df_val[LABEL_COLUMNS], df_test["text"], df_test[LABEL_COLUMNS]

In [9]:
import lightning as L
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AdamW
import torch
from datasets import Dataset
from torch.utils.data import DataLoader
from lightning.pytorch.loggers import WandbLogger
from torchmetrics.classification import MultilabelF1Score
from torchmetrics.classification import MultilabelAUROC
import torch.nn as nn

import numpy as np
from functools import partial
import os

In [10]:
def collate_fn(batch):
    # print("I am in collate_fn")
    input_ids = torch.stack([torch.LongTensor(sample["input_ids"]) for sample in batch])
    attention_mask = torch.stack([torch.Tensor(sample["attention_mask"]) for sample in batch])
    labels = [sample["labels"] for sample in batch]
    labels_tensor = [torch.tensor([0.0] * n_labels) for _ in labels]
    padded_tensors = []
    for label, label_tensor in zip(labels, labels_tensor):
        for l in label:
            label_tensor[int(l)] = 1.0

        padded_tensors.append(label_tensor.to(torch.float32))
    stacked_labels = torch.stack(padded_tensors)

    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': stacked_labels}

In [11]:
# step 1 - class of basic data structure

class MultiLabelBasicDataset(Dataset):
    def __init__(self, encodings):
        print("I am in MultiLabelDataset.__init__")
        self.encodings = encodings
        self.dataset_tokenized = Dataset.from_dict(self.encodings)
        self.labels = self.encodings["labels"]

In [12]:
# Step 2: All datasets (train, validation, and test)

class MultiLabelDataModule(L.LightningDataModule):
    def __init__(self, tokenizer, texts_train, labels_train, texts_val, labels_val, texts_test, labels_test, batch_size=batch_size, max_token_len=max_length):
        super().__init__()
        print("I am in MultiLabelDataModule.__init__()")
        self.tokenizer = tokenizer
        self.texts_train = texts_train
        self.labels_train = labels_train
        self.texts_val = texts_val
        self.labels_val = labels_val
        self.texts_test = texts_test
        self.labels_test = labels_test
        self.batch_size = batch_size
        self.max_token_len = max_token_len

    def tokenize(self, texts, labels):
        print("tokenize func")
        input_ids, attention_mask = [], []
        text_tokenized=self.tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=max_length)
        input_ids += text_tokenized["input_ids"]
        attention_mask += text_tokenized["attention_mask"]
        labels_np = labels.to_numpy().astype(np.float32)
        labels_indices = [np.where(row == 1)[0].astype(np.float32).tolist() for row in labels_np]

        ret = {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels_indices,
        }

        print("tokenize finished")
        return ret

    def setup(self, stage=None):
        # Split the dataset here if needed, for now assuming texts and labels are ready
        # Tokenize the text
        print("I am in setup")

        # Create dataset train
        print("Create dataset train")
        self.encodings_train = self.tokenize(self.texts_train, self.labels_train)
        self.dataset_train = MultiLabelBasicDataset(self.encodings_train).dataset_tokenized

        # Create dataset val
        print("Create dataset val")
        self.encodings_val = self.tokenize(self.texts_val, self.labels_val)
        self.dataset_val = MultiLabelBasicDataset(self.encodings_val).dataset_tokenized

        # Create dataset test
        print("Create dataset test")
        self.encodings_test = self.tokenize(self.texts_test, self.labels_test)
        self.dataset_test = MultiLabelBasicDataset(self.encodings_test).dataset_tokenized

    def train_dataloader(self):
        # Implement logic to split dataset if needed or use full dataset for training
        print("I am in train_dataloader")
        return DataLoader(self.dataset_train, batch_size=self.batch_size, shuffle=True, collate_fn=collate_fn, num_workers=num_workers)

    def val_dataloader(self):
        # Implement logic to split dataset if needed or use a portion for validation
        # Placeholder: using the same dataset for simplicity, replace with actual validation set
        print("I am in val_dataloader")
        return DataLoader(self.dataset_val, batch_size=self.batch_size, collate_fn=collate_fn, num_workers=num_workers)

    def test_dataloader(self):
        # Implement logic to split dataset if needed or use a portion for validation
        # Placeholder: using the same dataset for simplicity, replace with actual validation set
        print("I am in test_dataloader")
        return DataLoader(self.dataset_test, batch_size=self.batch_size, collate_fn=collate_fn, num_workers=num_workers)


In [13]:
# Step 3 - build the model

class MultiLabelDetectionModel(L.LightningModule):
    def __init__(self, model_name=model_name, n_labels = n_labels):
        print("I am in MultiLabelDetectionModel.__init__")
        super().__init__()
        self.save_hyperparameters()
        self.n_labels = n_labels
        print("I am after: save_hyperparameters")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        print("I am after: tokenizer init")
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=n_labels,
           id2label=id2label, label2id=label2id,
                       problem_type = "multi_label_classification")
        print("I am after: model init")
        self.criterion = nn.BCEWithLogitsLoss()
        print("I am after: nn.BCEWithLogitsLoss()")
        self.auroc = MultilabelAUROC(n_labels)
        print("I am after: MultilabelAUROC()")
        self.multi_label_f1 = MultilabelF1Score(n_labels)
        print("I am after: multi_label_f1")

        self.train_loss = []
        self.train_predictions = []
        self.train_labels = []
        self.val_loss = []
        self.val_predictions = []
        self.val_labels = []
        self.test_loss = []
        self.test_predictions = []
        self.test_labels = []

    def forward(self, input_ids, attention_mask):
        # print("I am in forward")
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        # print("outputs: ")
        # print(outputs)

        return outputs.logits

    def training_step(self, batch, batch_idx):
        # print("I am in training_step")
        input_ids, attention_mask, labels = batch["input_ids"], batch["attention_mask"], batch["labels"]
        logits = self(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.sigmoid(logits)  # Apply sigmoid to convert logits to probabilities
        loss = self.criterion(predictions, labels)
        self.train_loss.append(loss)
        # if self.train_loss:
        self.log("training/loss_epoch:", torch.stack(self.train_loss).mean(), on_step=False, on_epoch=True, logger=True)
        self.train_predictions.extend(predictions)
        self.train_labels.extend(labels)

        return loss

    def on_training_epoch_end(self, outputs):
        print("I am in on_training_epoch_end")
        print("outputs: ")
        print(outputs)
        if self.train_loss:
            avg_loss = torch.stack(self.train_loss).mean()
            self.log("training/loss_epoch:", avg_loss, prog_bar=True, logger=True)

        self.train_predictions = torch.stack(self.train_predictions)
        self.train_labels = torch.stack(self.train_labels).long()
        multi_label_f1_score = self.multi_label_f1(self.train_predictions, self.train_labels)
        print("training/multi label f1: ", multi_label_f1_score)
        self.log("training/multi label f1: ", multi_label_f1_score, prog_bar=True, logger=True)
        multi_label_auroc_score = self.auroc(self.train_predictions, self.train_labels)
        print("training/multi label auroc: ", multi_label_auroc_score)
        self.log("training/multi label auroc: ", multi_label_auroc_score, prog_bar=True, logger=True)

        self.train_loss = []
        self.train_predictions = []
        self.train_labels = []

    def validation_step(self, batch, batch_idx):
        # print("I am in validation_step")
        input_ids, attention_mask, labels = batch["input_ids"], batch["attention_mask"], batch["labels"]
        logits = self(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.sigmoid(logits)  # Apply sigmoid to convert logits to probabilities
        loss = self.criterion(predictions, labels)
        self.val_loss.append(loss)
        avg_loss = torch.stack(self.val_loss).mean()
        self.log("validation/loss_epoch:", avg_loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)

        self.val_predictions.extend(predictions)
        self.val_labels.extend(labels)

        return loss

    def on_validation_epoch_end(self):
        print("I am in on_validation_epoch_end")
        if self.val_loss:
            avg_loss = torch.stack(self.val_loss).mean()
            print("validation/loss_epoch:", avg_loss)
            self.log("validation/loss_epoch:", avg_loss, prog_bar=True, logger=True)

        self.val_predictions = torch.stack(self.val_predictions)
        self.val_labels = torch.stack(self.val_labels).long()
        multi_label_f1_score = self.multi_label_f1(self.val_predictions, self.val_labels)
        print("validation/multi label f1: ", multi_label_f1_score)
        self.log("validation/multi label f1: ", multi_label_f1_score, prog_bar=True, logger=True)
        multi_label_auroc_score = self.auroc(self.val_predictions, self.val_labels)
        print("validation/multi label auroc: ", multi_label_auroc_score)
        self.log("validation/multi label auroc: ", multi_label_auroc_score, prog_bar=True, logger=True)

        self.val_loss = []
        self.val_predictions = []
        self.val_labels = []

    def test_step(self, batch, batch_idx):
        # print("I am in test_step")
        input_ids, attention_mask, labels = batch["input_ids"], batch["attention_mask"], batch["labels"]
        logits = self(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.sigmoid(logits)  # Apply sigmoid to convert logits to probabilities
        loss = self.criterion(predictions, labels)
        self.log("test_loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.test_predictions.extend(predictions)
        self.test_labels.extend(labels)

        return loss

    def on_test_epoch_end(self):
        print("I am in on_test_epoch_end")
        if self.test_loss:
            avg_loss = torch.stack(self.test_loss).mean()
            print("test/loss_epoch:", avg_loss)
            self.log("test/loss_epoch:", avg_loss, prog_bar=True, logger=True)

        self.test_predictions = torch.stack(self.test_predictions)
        self.test_labels = torch.stack(self.test_labels).long()
        multi_label_f1_score = self.multi_label_f1(self.test_predictions, self.test_labels)
        print("test/multi label f1: ", multi_label_f1_score)
        self.log("test/multi label f1: ", multi_label_f1_score, prog_bar=True, logger=True)
        multi_label_auroc_score = self.auroc(self.test_predictions, self.test_labels)
        print("test/multi label auroc: ", multi_label_auroc_score)
        self.log("test/multi label auroc: ", multi_label_auroc_score, prog_bar=True, logger=True)

        self.test_loss = []
        self.test_predictions = []
        self.test_labels = []

    def predict(self, text):
        print("I am in predict")
        inputs = self.tokenizer(text, return_tensors='pt')

        rets = []
        with torch.no_grad():
            outputs = self.model(**inputs)
            logits = outputs.logits
            predictions = torch.sigmoid(logits)
            print("predictions: ")
            print(predictions)

            for i, prediction in enumerate(np.array(predictions[0])):
                if prediction >= 0.5:
                    rets.append(LABEL_COLUMNS[i].replace("_", " "))

            if len(rets) == 0:
                ret = "This message was approved"
            elif 1 == len(rets):
                ret = f"This message was {rets[0]}"
            elif 2 == len(rets):
                ret = f"This message was {rets[0]} and {rets[1]}"
            else:
                ret = f"This message was {rets[0]}, {rets[1]}, and {rets[2]}"

        return ret

    def configure_optimizers(self):
        print(f"I am in configure_optimizers - learning_rate: {learning_rate}")  # self.hparams.lr: {self.hparams.lr} -
        optimizer = torch.optim.Adam(params = self.parameters(), lr=learning_rate)

        return optimizer

In [14]:
# Download the model
#         Create instances of your model and data module, then initiate a trainer to start training:
#         python
#

# ***run only once***
model = MultiLabelDetectionModel()

I am in MultiLabelDetectionModel.__init__
I am after: save_hyperparameters


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


I am after: tokenizer init


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


I am after: model init
I am after: nn.BCEWithLogitsLoss()
I am after: MultilabelAUROC()
I am after: multi_label_f1


In [15]:
data_module = MultiLabelDataModule(tokenizer=model.tokenizer, texts_train=list(X_train), labels_train=y_train, texts_val=list(X_val), labels_val=y_val, texts_test=list(X_test), labels_test=y_test)

I am in MultiLabelDataModule.__init__()


In [16]:
wandb_logger = WandbLogger(project=wandb_project, name=wandb_name, group="yoav_yosef")

In [17]:
trainer = L.Trainer(max_epochs=epochs, devices=1 if torch.cuda.is_available() else 0, accelerator="auto", logger=wandb_logger)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [18]:
trainer.fit(model, data_module)

INFO: You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:lightning.pytorch.utilities.rank_zero:You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiati

I am in setup
Create dataset train
tokenize func
tokenize finished
I am in MultiLabelDataset.__init__
Create dataset val
tokenize func
tokenize finished
I am in MultiLabelDataset.__init__
Create dataset test
tokenize func


INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


tokenize finished
I am in MultiLabelDataset.__init__


INFO: 
  | Name           | Type                                | Params
-----------------------------------------------------------------------
0 | model          | DistilBertForSequenceClassification | 67.0 M
1 | criterion      | BCEWithLogitsLoss                   | 0     
2 | auroc          | MultilabelAUROC                     | 0     
3 | multi_label_f1 | MultilabelF1Score                   | 0     
-----------------------------------------------------------------------
67.0 M    Trainable params
0         Non-trainable params
67.0 M    Total params
267.823   Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name           | Type                                | Params
-----------------------------------------------------------------------
0 | model          | DistilBertForSequenceClassification | 67.0 M
1 | criterion      | BCEWithLogitsLoss                   | 0     
2 | auroc          | MultilabelAUROC                     | 0     
3 | m

...

Validation: |          | 0/? [00:00<?, ?it/s]

I am in on_validation_epoch_end
validation/loss_epoch: tensor(0.6312, device='cuda:0')
validation/multi label f1:  tensor(0.7894, device='cuda:0')
validation/multi label auroc:  tensor(0.9196, device='cuda:0')


Validation: |          | 0/? [00:00<?, ?it/s]

I am in on_validation_epoch_end
validation/loss_epoch: tensor(0.6339, device='cuda:0')
validation/multi label f1:  tensor(0.7764, device='cuda:0')
validation/multi label auroc:  tensor(0.9197, device='cuda:0')


INFO: `Trainer.fit` stopped: `max_epochs=10` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


In [19]:
test_loader = DataLoader(data_module.dataset_test, batch_size=batch_size, collate_fn=collate_fn)
trainer.test(model, test_loader)

I am in setup
Create dataset train
tokenize func
tokenize finished
I am in MultiLabelDataset.__init__
Create dataset val
tokenize func
tokenize finished
I am in MultiLabelDataset.__init__
Create dataset test
tokenize func


INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


tokenize finished
I am in MultiLabelDataset.__init__


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

I am in on_test_epoch_end
test/multi label f1:  tensor(0.7906, device='cuda:0')
test/multi label auroc:  tensor(0.9255, device='cuda:0')


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│  test/multi label auroc:  │    0.9255098104476929     │
│   test/multi label f1:    │    0.7906321883201599     │
│         test_loss         │    0.6291943788528442     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.6291943788528442,
  'test/multi label f1: ': 0.7906321883201599,
  'test/multi label auroc: ': 0.9255098104476929}]

In [20]:
model.predict("Hi!")

I am in predict
predictions: 
tensor([[0.0002, 0.0003, 0.0023]])


'This message was approved'

In [21]:
model.predict("White is better than black")

I am in predict
predictions: 
tensor([[0.0006, 0.2545, 0.0096]])


'This message was approved'

In [22]:
model.predict("White people are better than black people!")

I am in predict
predictions: 
tensor([[0.0018, 0.8575, 0.2598]])


'This message was hate speech'

In [23]:
model.predict("I hate every one!")

I am in predict
predictions: 
tensor([[9.5356e-04, 9.0282e-04, 9.8826e-01]])


'This message was bullying'

In [24]:
model.predict("The Jews control the world, they should be killed!")

I am in predict
predictions: 
tensor([[0.0025, 0.6745, 0.9725]])


'This message was hate speech and bullying'

In [35]:
model.predict("Jews")

I am in predict
predictions: 
tensor([[0.0418, 0.9606, 0.1251]])


'This message was hate speech'

In [36]:
model.predict("Shut up!")

I am in predict
predictions: 
tensor([[0.0031, 0.0012, 0.9972]])


'This message was bullying'

In [25]:
model.predict("Shut up nigger!")

I am in predict
predictions: 
tensor([[0.9703, 0.9984, 0.9936]])


'This message was nsfw, hate speech, and bullying'

In [26]:
model.predict("I will kick the shit out of you, retarded")  # un balanced data set

I am in predict
predictions: 
tensor([[0.9958, 0.0022, 0.9996]])


'This message was nsfw and bullying'

In [27]:
model.predict("Do you want a bigger penis?")  # un balanced data set

I am in predict
predictions: 
tensor([[9.9008e-01, 1.1481e-04, 5.6245e-02]])


'This message was nsfw'

In [28]:
model.predict("Shut up you piece of shit!")

I am in predict
predictions: 
tensor([[9.9596e-01, 5.4021e-04, 9.9925e-01]])


'This message was nsfw and bullying'

In [29]:
model.predict("Watch free porn now!")

I am in predict
predictions: 
tensor([[6.7626e-04, 7.3903e-04, 8.7760e-01]])


'This message was bullying'

In [39]:
model.predict("I love women with huge boobs!")

I am in predict
predictions: 
tensor([[7.8944e-04, 1.4018e-03, 9.4614e-01]])


'This message was bullying'

In [40]:
model.predict("I love all women!")

I am in predict
predictions: 
tensor([[4.4899e-04, 2.4594e-03, 9.6057e-01]])


'This message was bullying'

In [30]:
model.predict("You are a nigger!")

I am in predict
predictions: 
tensor([[0.9376, 0.9977, 0.9892]])


'This message was nsfw, hate speech, and bullying'

In [31]:
torch.save(model, f"/content/drive/MyDrive/BIU - Final Project/Models/{task_folder}/second run/model_multi_label.ckpt")

In [32]:
import pickle
from google.colab import files

pickle.dump(model.model, open(f"/content/drive/MyDrive/BIU - Final Project/Models/{task_folder}/second run/model_{pickle_name}.pkl", "wb"))
pickle.dump(model.tokenizer, open(f"/content/drive/MyDrive/BIU - Final Project/Models/{task_folder}/second run/tokenizer_{pickle_name}.pkl", "wb"))


In [33]:
import torch

# Save the model
torch.save(model.model.state_dict(), f"/content/drive/MyDrive/BIU - Final Project/Models/{task_folder}/second run//model_{pickle_name}.pth")

In [34]:
model.tokenizer.save_pretrained(f"/content/drive/MyDrive/BIU - Final Project/Models/{task_folder}/second run//tokenizer_{pickle_name}.pth")

('/content/drive/MyDrive/BIU - Final Project/Models/Multi Label Hate Speech Bullying NSFW/second run//tokenizer_kaggle - distilbert-base-uncased_multi_label_transformer_train_sample_balanced toxic data set_epochs_10_batch_size_32_max_length_500_learning_rate_1e-05_num_workers_15_with_all_accuracy.pth/tokenizer_config.json',
 '/content/drive/MyDrive/BIU - Final Project/Models/Multi Label Hate Speech Bullying NSFW/second run//tokenizer_kaggle - distilbert-base-uncased_multi_label_transformer_train_sample_balanced toxic data set_epochs_10_batch_size_32_max_length_500_learning_rate_1e-05_num_workers_15_with_all_accuracy.pth/special_tokens_map.json',
 '/content/drive/MyDrive/BIU - Final Project/Models/Multi Label Hate Speech Bullying NSFW/second run//tokenizer_kaggle - distilbert-base-uncased_multi_label_transformer_train_sample_balanced toxic data set_epochs_10_batch_size_32_max_length_500_learning_rate_1e-05_num_workers_15_with_all_accuracy.pth/vocab.txt',
 '/content/drive/MyDrive/BIU - F